# Playground for ASFPy module development

In [1]:
import os
import csv
import random
from copy import deepcopy

from asfpy import asfpy

In [2]:
from pathlib import Path

FOLDER = "asfp2020"
FILE = "processed-editors-list-draft.csv"

path = Path("__file__").parent.absolute()

filename = path.parent.joinpath(FOLDER).joinpath(FILE)

editors = asfpy.read_preprocessed_editors_list_csv(filename)

In [5]:
applicants = [
    {
        "id": "APP001",
        "urm": False,
        "lim": False,
        "du": True,
        "flexible": True,
        "conflicts": {"Peter Sokol-Hessner"},
        "categories": {
            "Cognitive Psychology"
        },
    },
    {
        "id": "APP002",
        "urm": True,
        "lim": False,
        "du": True,
        "flexible": True,
        "conflicts": {"Peter Sokol-Hessner"},
        "categories": {
            "Cognitive Psychology"
        },
    },
    {
        "id": "APP003",
        "urm": True,
        "lim": False,
        "du": True,
        "flexible": True,
        "conflicts": {"Peter Sokol-Hessner", "Tim Sweeny"},
        "categories": {
            "Social Psychology",
            "Cognitive Psychology"
        }
    },
    {
        "id": "APP004",
        "urm": False,
        "lim": False,
        "du": True,
        "flexible": True,
        "conflicts": {"Michelle Rozenman"},
        "categories": {
            "Clinical Psychology"
        },
    },
    {
        "id": "APP005",
        "urm": True,
        "lim": True,
        "du": True,
        "flexible": True,
        "conflicts": {"Michelle Rozenman"},
        "categories": {
            "Cognitive Psychology",
            "Affective Psychology"
        },
    },
    {
        "id": "APP006",
        "urm": True,
        "lim": False,
        "du": True,
        "flexible": True,
        "conflicts": {"Michelle Rozenman"},
        "categories": {
            "Affective Psychology",
            "Cognitive Psychology"
        }
    },
    {
        "id": "APP007",
        "urm": True,
        "lim": True,
        "du": False,
        "flexible": False,
        "conflicts": {"Michelle Rozenman"},
        "categories": {
            "Clinical Psychology"
        }
    },
    {
        "id": "APP008",
        "urm": False,
        "lim": True,
        "du": False,
        "flexible": True,
        "conflicts": {"Michelle Rozenman"},
        "categories": {
            "Developmental Psychology"
        }
    },
    {
        "id": "APP009",
        "urm": True,
        "lim": True,
        "du": False,
        "flexible": False,
        "conflicts": {"Michelle Rozenman"},
        "categories": {
            "Clinical Psychology"
        }
    },
    {
        "id": "APP010",
        "urm": False,
        "lim": True,
        "du": False,
        "flexible": True,
        "conflicts": {"Michelle Rozenman"},
        "categories": {
            "Affective Psychology"
        }
    },
]

In [10]:
_editors = deepcopy(editors)
_applicants = deepcopy(asfpy.randomize(applicants))

p_applicants = asfpy.prioritize(_applicants, rank_method = asfpy.asfp_rank)

allocation_output = asfpy.allocate(p_applicants, _editors)

# Allocation Output

The output from running the allocation method includes a list of matchings, a list of unmatched applicants by identifier strings, and a list of editors _after_ decrementing capacities.

In [14]:
allocation_output["matchings"]

[{'applicant': 'APP005', 'editors': ['EDI_013', 'EDI_015']},
 {'applicant': 'APP007', 'editors': ['EDI_012', 'EDI_008']},
 {'applicant': 'APP009', 'editors': ['EDI_016', 'EDI_008']},
 {'applicant': 'APP003', 'editors': ['EDI_021', 'EDI_015']},
 {'applicant': 'APP006', 'editors': ['EDI_013', 'EDI_015']},
 {'applicant': 'APP002', 'editors': ['EDI_024', 'EDI_015']},
 {'applicant': 'APP010', 'editors': ['EDI_013', 'EDI_004']},
 {'applicant': 'APP008', 'editors': ['EDI_042', 'EDI_023']},
 {'applicant': 'APP004', 'editors': ['EDI_012', 'EDI_008']},
 {'applicant': 'APP001', 'editors': ['EDI_024', 'EDI_015']}]

In [15]:
allocation_output["unmatched"]

[]

In [16]:
allocation_output["editors"]

[{'name': 'Jill Holm-Denoma',
  'email': 'jholmden@du.edu',
  'role': 'faculty',
  'capacity': 2,
  'categories': {'Clinical Psychology'},
  'id': 'EDI_001'},
 {'name': 'Melissa Brewster',
  'email': 'Melissa.Brewster@du.edu',
  'role': 'student',
  'capacity': 3,
  'categories': {'Clinical Psychology', 'Developmental Psychology'},
  'id': 'EDI_002'},
 {'name': 'Emily Jones',
  'email': 'emily.l.jones@du.edu',
  'role': 'student',
  'capacity': 3,
  'categories': {'Clinical Psychology'},
  'id': 'EDI_003'},
 {'name': 'Özlü Aran',
  'email': 'ozlu.aran@du.edu',
  'role': 'student',
  'capacity': 5,
  'categories': {'Affective Psychology',
   'Developmental Psychology',
   'Social Psychology'},
  'id': 'EDI_004'},
 {'name': 'Paige Lloyd',
  'email': 'Paige.lloyd@du.edu',
  'role': 'faculty',
  'capacity': 6,
  'categories': {'Affective Psychology', 'Social Psychology'},
  'id': 'EDI_005'},
 {'name': 'Kevin Summers',
  'email': 'kevin.summers@du.edu',
  'role': 'student',
  'capacity': 7,

In [ ]:
set(applicants)